In [1]:
!git clone https://github.com/buitrongtrinh/ImplementAutoencoderByCuda.git

Cloning into 'ImplementAutoencoderByCuda'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 52 (delta 11), reused 27 (delta 7), pack-reused 21 (from 1)
Receiving objects: 100% (52/52), 162.43 MiB | 21.89 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Updating files: 100% (31/31), done.


In [2]:
%cd ImplementAutoencoderByCuda/

/content/ImplementAutoencoderByCuda


## PHRASE 2

In [3]:
%cd Phrase2/
!nvcc ./*.cu -o phrase2 -std=c++11 -O3 -arch=sm_75

/content/ImplementAutoencoderByCuda/Phrase2


In [4]:
!./phrase2

========== GPU AUTOENCODER TRAINING ==========
Batch size = 64
Epochs     = 20
LR         = 0.001
Loading CIFAR-10 dataset...
Dataset loaded successfully!
Training samples: 50000
Test samples: 10000
[GPUAutoencoder] Initialized (Phase 2.1 – standalone)
Training ....... 
[Epoch 0]
=== Batch 0/782 | Loss = 0.26484
^C


## Phrase3

In [5]:
%cd ..
%cd Phrase3
!nvcc dataset.cu gpu_autoencoder.cu kernels_backward.cu kernels_forward.cu main.cu -o phrase3 -std=c++11 -O3 -arch=sm_75

/content/ImplementAutoencoderByCuda
/content/ImplementAutoencoderByCuda/Phrase3


In [6]:
!./phrase3

========== GPU AUTOENCODER TRAINING ==========
Batch size = 64
Epochs     = 20
LR         = 0.001
Loading CIFAR-10 dataset...
Dataset loaded successfully!
Training samples: 50000
Test samples: 10000
[GPUAutoencoder] Initialized (Phase 2.1 – standalone)
Training ....... 
[Epoch 0]
=== Batch 0/782 | Loss = 0.301698
=== Batch 100/782 | Loss = 0.262242
^C


In [ ]:
!nvcc ex_fea.cu dataset.cu gpu_autoencoder.cu kernels_forward.cu kernels_backward.cu -o extract_feature -std=c++11 -O3 -arch=sm_75

In [ ]:
!./extract_feature

========== FEATURE EXTRACTION ==========
Loading CIFAR-10 dataset...
Dataset loaded successfully!
Training samples: 50000
Test samples: 10000
[GPUAutoencoder] Initialized (Phase 2.1 – standalone)
[INFO] Weights loaded successfully: autoencoder_gpu.weights
[Extract] TRAIN
Train batch 0/782
Train batch 100/782
Train batch 200/782
Train batch 300/782
Train batch 400/782
Train batch 500/782
Train batch 600/782
Train batch 700/782
[Extract] TEST
Test batch 0/157
Test batch 50/157
Test batch 100/157
Test batch 150/157
========== EXTRACTION DONE ==========
[GPUAutoencoder] Destroyed


In [ ]:
!zip features.zip train_features*.bin train_labels*.bin test_features.bin test_labels.bin

  adding: train_features.bin (deflated 100%)
  adding: train_labels.bin (deflated 84%)
  adding: test_features.bin (deflated 100%)
  adding: test_labels.bin (deflated 83%)


In [ ]:
from google.colab import files
files.download("features.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>